## How many of the Top 200 songs occur on both the US and Global charts? 

Timeframe: past 52 weeks

Observations:
1. 829 songs are common across both charts
 - This comprises 64% of the US chart and 72% of the Global chart <br>
<br> 
- Although the US has fewer common songs in the Top 200, the common songs comprise a large percent of streams
 - Common songs comprise 90% of streams in the US and 84% of streams globally
 - Question:  Are there songs on either chart with high streams that do not cross over to the other chart?
 - Question:  If there are such songs, what differentiates them from songs that cross over?<br>
<br>
- There appear to be two different types of songs that are common to both charts:
 - Those with a high correlation of weeks in the chart (globally popular) and those with no correlation
 - Question: What are the attributes that distinguish the two subsets?

Additional questions:
- Is there a time factor indicating direction of influence (timing of when common songs show up on one chart vs the other)?


In [ ]:
# Dependencies and 
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
from datetime import datetime, timedelta
from scipy.stats import linregress

# data files
US_path = "Combined CSVs/US_combined.csv"
Global_path = "Combined CSVs/Global_combined.csv"

# Read the mouse data and the study results
us_data = pd.read_csv(US_path)
global_data = pd.read_csv(Global_path)


In [ ]:
# create unique Artist Track field and display column headers in US data
us_data["ArtistTrack"] = us_data["Artist"] + us_data["Track Name"]
us_data.head()

In [ ]:
# temp - value counts to determine if there are headers in the data rows
us_data["Position"].value_counts()

In [ ]:
# temp
# drop headers in data and convert Streams to integer - US data
#
us_data.drop(us_data[us_data.Position == "Position"].index, inplace=True)
us_data["Position"] = us_data["Position"].astype(int)
us_data["Streams"] = us_data["Streams"].astype(int)
us_data["Artist"] = us_data["Artist"].str.replace('$','S')
us_data["Track Name"] = us_data["Track Name"].str.replace('$','S')

print (f"Records in US data after dropping duplicate headers: {len(us_data)}")
us_data.dtypes

In [ ]:
# temp - display record counts and data types of rows in US data
print (f"Records in Global data: {len(global_data)}")
global_data.dtypes

In [ ]:
# create unique Artist Track field and display column headers in Global data
global_data["ArtistTrack"] = global_data["Artist"] + global_data["Track Name"]
global_data.head()

In [ ]:
# temp - value counts to determine if there are headers in the data rows
global_data["Position"].value_counts()

In [ ]:
# temp
# drop headers in data and convert Streams to integer - Global data
global_data.drop(global_data[global_data.Position == "Position"].index, inplace=True)
global_data["Position"] = global_data["Streams"].astype(int)
global_data["Streams"] = global_data["Streams"].astype(int)
global_data["Artist"] = global_data["Artist"].str.replace('$','S')
global_data["Track Name"] = global_data["Track Name"].str.replace('$','S')


print (f"Records in US data after dropping duplicate headers: {len(global_data)}")
global_data.dtypes

In [ ]:
print("Input counts:")
us_data_recs_in = len(us_data)
global_data_recs_in = len(global_data)
print(f"rows in us: {us_data_recs_in}")
print(f"rows in global: {global_data_recs_in}")

us_dates_in = us_data["Date"].value_counts()
global_dates_in = global_data["Date"].value_counts()
print(f"unique dates in us: {len(us_dates_in)}")
print(f"unique dates in global: {len(global_dates_in)}")
     

In [ ]:
# get current date and format to compare to date from input (mm.dd.yy)

#
# Don't think I need this cell, but keep for the time being - pull into reference do for future use
#


full_year = datetime.today().strftime('%Y')
month_day = datetime.today().strftime('%m.%d')

# examples for confirming how string parsing works - keep in place for the time being
month_day2 = month_day[2:]
month_day3 = month_day[:3]
#print (f"month_day: {month_day}")
#print (f"month_day2: {month_day2}")
#print (f"month_day3: {month_day3}")

todays_date = f"{month_day}.{full_year[2:]}"
print (todays_date)

In [ ]:
# Get date to use for pulling last 2 months of data
#print(datetime.today())
print (datetime.today() + timedelta(days=-60))

compare_year = (datetime.today() + timedelta(days=-60)).strftime('%Y')
compare_month_day = (datetime.today() + timedelta(days=-60)).strftime('%m.%d')

compare_date = f"{compare_year[2:]}.{compare_month_day}"
print(compare_date)


In [ ]:
# reformat date from input for sorting/comparison

# US Data
us_data["year"] = [x.strip()[-2:] for x in us_data['Date']]
# strip [5:] does the same thing as strip [-2:] with the Date field, use [:5] to pull first 5 characters
# keep for now but move to reference doc for future use
#us_data["month_day"] = [x.strip()[5:] for x in us_data['Date']]
us_data["month_day"] = [x.strip()[:5] for x in us_data['Date']]
us_data["reformatted date"] = us_data["year"] + "." + us_data["month_day"]

# Global Data
global_data['Date'] = global_data['Date'].astype(str)
global_data["year"] = [x.strip()[-2:] for x in global_data['Date']]
global_data["month_day"] = [x.strip()[:5] for x in global_data['Date']]
global_data["reformatted date"] = global_data["year"] + "." + global_data["month_day"]

global_data.tail()

In [ ]:
print("Counts after date conversion:")
print("")

us_data_recs_chk1 = len(us_data)
global_data_recs_chk1 = len(global_data)
print(f"rows in us: {us_data_recs_chk1}")
print(f"rows in global: {global_data_recs_chk1}")

us_dates_chk1 = us_data["Date"].value_counts()
global_dates_chk1 = global_data["Date"].value_counts()
print(f"unique dates in us: {len(us_dates_chk1)}")
print(f"unique dates in global: {len(global_dates_chk1)}")



In [ ]:
# sanity check

us_recs_aft_cnv = len(us_data["Date"].value_counts())
global_recs_aft_cnv = len(global_data["Date"].value_counts())

#print(f"unique dates in us after date conversion: {len(len(us_data["Date"].value_counts())}")
print(f"unique dates in us after conversion: {us_recs_aft_cnv}")
print(f"unique dates in global after conversion: {global_recs_aft_cnv}")

us_data_recs_1 = len(us_data)
global_data_recs_1 = len(global_data)
print(f"rows in us: {us_data_recs_1}")
print(f"rows in global: {global_data_recs_1}")

us_dates_1 = us_data["Date"].value_counts()
global_dates_1 = global_data["Date"].value_counts()
print(f"unique dates in us: {len(us_dates_1)}")
print(f"unique dates in global: {len(global_dates_1)}")

In [ ]:
# Identify unique song/artists from the past 2 months: US & Global
# For each unique song/artist identify min & max position and total streams
# What's the total population of unique song/artists in each dataset?
# How many unique song/artists are common between datasets?
# What percent of total dataset do the common song/artists comprise?
# Does statistical analysis of the datasets provide insight?

In [ ]:
# create summary dataframes for comparing US to Global

us_groupby = us_data.groupby(["ArtistTrack"])
us_track_df = pd.DataFrame(us_groupby["Artist"].max())
us_track_df["Track Name"] = us_groupby["Track Name"].max()
us_track_df["Weeks in Top 200"] = us_groupby["Position"].count()
us_track_df["Top Position"] = us_groupby["Position"].min()
us_track_df["Low Position"] = us_groupby["Position"].max()
us_track_df["Total Streams"] = us_groupby["Streams"].sum()

global_groupby = global_data.groupby(["ArtistTrack"])
global_track_df = pd.DataFrame(global_groupby["Artist"].max())
global_track_df["Track Name"] = global_groupby["Track Name"].max()
global_track_df["Weeks in Top 200"] = global_groupby["Position"].count()
global_track_df["Top Position"] = global_groupby["Position"].min()
global_track_df["Low Position"] = global_groupby["Position"].max()
global_track_df["Total Streams"] = global_groupby["Streams"].sum()

global_track_df.head()

In [ ]:
# create dataframe of songs common to both charts
common_track_df = pd.merge(us_track_df, global_track_df, how='inner', on='ArtistTrack', suffixes=('_US', '_GL'))
common_track_df.head()

In [ ]:
def print_scatterchart(x_axis, y_axis, title, x_label, y_label):
    
    fig = plt.figure()
    plt.scatter(x_axis, y_axis, marker="o", facecolors="darkblue")
    fig.suptitle(title, fontsize=12)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.show()

In [ ]:
def print_scatter_and(x_axis, y_axis, title, x_label, y_label):
# print scatter charter and regression analsysi

    # make x_axis amenable to calculation of regression values
    x_axis = np.asarray(x_axis)
    
    # run linregress to get components for correlaton coefficient and regression model
    (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis,  y_axis)
    regress_values = x_axis * slope + intercept

    # calc correlation coefficient and format regression model equation
    correlation_coefficient = round(rvalue**2, 2)
    line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

    # figure out where to place the regression model equation
    test_val = (slope*max(x_axis) + intercept) - (slope*min(x_axis) + intercept)
    x_pos = min(x_axis)
    if test_val > 0:
        if max(y_axis) > 25:
            y_pos = max(y_axis) - 5
        else:
            y_pos = max(y_axis) - 1
    else:
        if max(y_axis) > 25:
            y_pos = min(y_axis) + 5
        else:
            y_pos = min(y_axis) + 1

    # format and print plot
    fig = plt.figure()
    plt.scatter(x_axis, y_axis, marker="o", facecolors="#79abcd")
    plt.plot(x_axis,regress_values,"#f70d30")
    plt.annotate(line_eq,(x_pos,y_pos),fontsize=15,color="#f70d30",fontweight="bold")
    fig.suptitle(title, fontsize=12)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.show()
    print(f'Correlation coefficient = {correlation_coefficient}')
    print('')

In [ ]:
# calculate total streams
one_billion = 1000000000
one_million = 1000000

# calculate raw totals
total_streams_US = us_track_df["Total Streams"].sum()
total_streams_GL = global_track_df["Total Streams"].sum()
total_common_streams_US = common_track_df["Total Streams_US"].sum()
total_common_streams_GL = common_track_df["Total Streams_GL"].sum()
total_unique_streams_US = total_streams_US - total_common_streams_US
total_unique_streams_GL = total_streams_GL - total_common_streams_GL

# calculate print totals
# US total streams
if total_streams_US > one_billion:
    denominator = one_billion
    stream_label_US = "billion"
else:    
    denominator = one_million
    stream_label_US = "million"

total_streams_US_f = float("{:.1f}".format(total_streams_US / denominator))
common_streams_US_f = float("{:.1f}".format(total_common_streams_US / denominator))
total_unique_streams_US_f = float("{:.1f}".format(total_unique_streams_US / denominator))


# Global total streams
if total_streams_GL > one_billion:
    denominator = one_billion
    stream_label_GL = "billion"
else:    
    denominator = one_million
    stream_label_GL = "million"

total_streams_GL_f = float("{:.1f}".format(total_streams_GL / denominator))
common_streams_GL_f = float("{:.1f}".format(total_common_streams_GL / denominator))
total_unique_streams_GL_f = float("{:.1f}".format(total_unique_streams_GL / denominator))

streams_bars_df = pd.DataFrame({"US":[total_common_streams_US, total_unique_streams_US],
                                "Global":[total_common_streams_GL, total_unique_streams_GL]})

#print(total_streams_GL)
#print(total_streams_GL_f)
#print(total_common_streams_GL)
#print(common_streams_GL_f)
#print(total_unique_streams_GL)
#print(total_unique_streams_GL_f)
#streams_bars_df.dtypes


In [ ]:
# calculate song counts
total_songs_US = len(us_track_df)
total_songs_GL = len(global_track_df)
total_songs_common = len(common_track_df)

total_unique_songs_US = total_songs_US - total_songs_common
total_unique_songs_GL = total_songs_GL - total_songs_common

print(total_unique_songs_GL)
print(total_songs_common)
print(total_songs_GL)


In [ ]:
# calculate percentages
common_streams_pct_US = float("{:.1f}".format((total_common_streams_US / total_streams_US) * 100))
unique_streams_pct_US = float("{:.1f}".format(100 - common_streams_pct_US))

common_streams_pct_GL = float("{:.1f}".format((total_common_streams_GL / total_streams_GL) * 100))
unique_streams_pct_GL = float("{:.1f}".format(100 - common_streams_pct_GL))

common_songs_pct_US = float("{:.1f}".format((total_songs_common / total_songs_US) * 100))
unique_songs_pct_US = float("{:.1f}".format(100 - common_songs_pct_US))

common_songs_pct_GL = float("{:.1f}".format((total_songs_common / total_songs_GL) * 100))
unique_songs_pct_GL = float("{:.1f}".format(100 - common_songs_pct_GL))

In [ ]:
print(f"The US Top 200 chart contains {total_songs_US} songs totaling {total_streams_US_f} {stream_label_US} streams")
print(f"The Global Top 200 chart contains {len(global_track_df)} songs totaling {total_streams_GL_f} {stream_label_GL} streams")
print(f"{total_songs_common} songs are common across both charts")
print("")

Artist_with_most_songs = common_track_df["Artist_US"].mode()
if len(Artist_with_most_songs) == 1:
    ArtistX = Artist_with_most_songs[0]
    SongsX_df = common_track_df.loc[common_track_df["Artist_US"] == Artist_with_most_songs[0]]
    print(f"The artist with the most songs on both charts is {Artist_with_most_songs[0]} with {len(SongsX_df)} songs")

# Identify songs with longest run in each chart
longest_run_US = common_track_df["Weeks in Top 200_US"].max()
longest_run_GL = common_track_df["Weeks in Top 200_GL"].max()
longest_run_songs_US_df = common_track_df.loc[common_track_df["Weeks in Top 200_US"] == longest_run_US]
longest_run_songs_GL_df = common_track_df.loc[common_track_df["Weeks in Top 200_GL"] == longest_run_GL]

print(f"There were {len(longest_run_songs_US_df)} songs that stayed in the US Top 200 for {longest_run_US} weeks")
print(f"There were {len(longest_run_songs_GL_df)} songs that stayed in the Global Top 200 for {longest_run_US} weeks")

common_longest_df = pd.merge(longest_run_songs_US_df, longest_run_songs_GL_df, how='inner', on='ArtistTrack')
len(common_longest_df)

if longest_run_US > longest_run_GL:
    print(f"There were {len(common_longest_df)} songs that stayed in the both charts for {longest_run_US} weeks")
else:
    print(f"There were {len(common_longest_df)} songs that stayed in the both charts for {longest_run_GL} weeks")

# Weeks on US chart vs Weeks on Global chart - common songs
print_scatter_and(common_track_df["Weeks in Top 200_US"], common_track_df["Weeks in Top 200_GL"], "Weeks on Chart: US vs Global", "Weeks on US Chart", "Weeks on Global Chart")
print("")

In [ ]:
# Observation:  Correlation of weeks on chart for common songs in general is not strong
#               but there appears to be a subset of that have a very high correlation    
# Question: Are there common attributes of the subset?  Could these be indicators of a song crossing over to another chart?
#
# Observation:  There are songs that have little to no correlation
# Question: Are there attributes that indicate that a song will not cross over to another chart (e.g, language)
# 

In [ ]:
# Compare US Streams to Global Songs
songs_bars_df = pd.DataFrame({"Common":[common_songs_pct_US, common_songs_pct_GL],
                              "Unique":[unique_songs_pct_US, unique_songs_pct_GL]})
labels = ["US","Global"]
ax = songs_bars_df.plot.bar(stacked=True,title="Composition of US Songs vs Global Songs")
ax.set_xticklabels(labels,rotation=45)
ax.set_ylabel("Percent")

print(f"US common songs {common_songs_pct_US}%, unique songs {unique_songs_pct_US}%")
print(f"Global common songs {common_songs_pct_GL}%, unique songs {unique_songs_pct_GL}%")


In [ ]:
# Compare US Streams to Global Streams
streams_bars_df = pd.DataFrame({"Common":[common_streams_pct_US, common_streams_pct_GL],
                                "Unique":[unique_streams_pct_US, unique_streams_pct_GL]})
labels = ["US","Global"]
ax = streams_bars_df.plot.bar(stacked=True,title="Composition of US Streams vs Global Streams")
ax.set_xticklabels(labels,rotation=45)
ax.set_ylabel("Percent")

print(f"US common streams {common_streams_pct_US}%, unique streams {unique_streams_pct_US}%")
print(f"Global common streams {common_streams_pct_GL}%, unique streams {unique_streams_pct_GL}%")


In [ ]:
# Observation:  US songs comprise a smaller percent of songs in the Top 200, but a larger percent of streams
# Question: How valid an indicator is stream counts that a song from the US Top 200 will appear in the Global Top 200?
#           Of songs in the US Top 200 with high stream counts, how many cross into the Global Top 200?
# Question: Is there and observable time factor between when a common song appears on one vs the other chart?
#           Who is the influencer?
